In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine, text, select
from sqlalchemy.orm import sessionmaker
import os
from dotenv import load_dotenv

In [2]:
# Loading env variables
load_dotenv(dotenv_path='.\.env')

user = os.getenv('user')
password = os.getenv('password')
host = os.getenv('host')
port = os.getenv('port')
database = os.getenv('database')

In [3]:
# Creating Postgre SQL connection, passing user and password as arguments when running script
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{database}')

In [4]:
# Importing the 4 tables
df_sales = pd.read_sql_query("SELECT * FROM fact_sales", con=engine, parse_dates='transaction_date')
df_customers = pd.read_sql_query("SELECT * FROM dim_customers", con=engine)
df_products = pd.read_sql_query("SELECT * FROM dim_products", con=engine)
df_state_region = pd.read_sql_query("SELECT * FROM state_region_mapping", con=engine)

In [5]:
def df_details(df:pd.DataFrame) -> None:
    '''Function to provide main details on a DataFrame'''
    
    print(f'DataFrame Shape: {df.shape}\n')
        
    details_df = pd.DataFrame({'Data Type':df.dtypes, 
                               'Count': df.count(),
                               'Missing':df.isnull().sum(),
                               'Missing (%)':round((df.isnull().sum())/df.shape[0]*100,2),
                               'Mean':round(df.mean(numeric_only=True), 0),
                               'Min': round(df.min(numeric_only=True),0),
                               'Median': round(df.median(numeric_only=True), 0),
                               'Max': round(df.max(numeric_only=True),0),
                               '# Unique': df.nunique()},
                                index=df.columns)
    
    return details_df

### Sales Table

In [6]:
df_sales.head()

,transaction_date,customer_id,description,stock_code,invoice_no,quantity,sales,unit_price
0,2020-12-01 08:26:00,17850.0,Dog and Puppy Pads,85123A,536365.0,1,15.30,15.30
1,2020-12-01 08:34:00,13047.0,Pet Odor Eliminator,84879,536367.0,6,60.84,10.14
2,2020-12-01 08:34:00,13047.0,Rechargeable Pet Nail Grinder,22960,536368.0,1,25.50,25.50
3,2020-12-01 09:00:00,13748.0,Litter Slide Multi-Cat Scented Clumping Clay C...,22086,536371.0,14,214.20,15.30
4,2020-12-01 09:02:00,17850.0,Dog and Puppy Pads,85123A,536373.0,1,15.30,15.30


In [7]:
sales_details = df_details(df_sales)
sales_details

DataFrame Shape: (25065, 8)



,Data Type,Count,Missing,Missing (%),Mean,Min,Median,Max,# Unique
transaction_date,datetime64[ns],25065,0,0.00,NaN,NaN,NaN,NaN,11408
customer_id,float64,20649,4416,17.62,15289.0,12347.0,15152.0,18287.0,3146
description,object,25065,0,0.00,NaN,NaN,NaN,NaN,21
stock_code,object,25065,0,0.00,NaN,NaN,NaN,NaN,20
invoice_no,float64,24404,661,2.64,559953.0,536365.0,560224.0,581585.0,11427
quantity,int64,25065,0,0.00,3.0,-321.0,1.0,717.0,108
sales,float64,25065,0,0.00,61.0,-4911.0,30.0,38970.0,610
unit_price,float64,25065,0,0.00,29.0,0.0,24.0,3897.0,101


In [8]:
df_sales.transaction_date.min(), df_sales.transaction_date.max()

(Timestamp('2020-12-01 08:26:00'), Timestamp('2021-12-09 12:31:00'))

<b>SALES OBSERVATIONS:</b>

- transaction_date:  Range is from Dec 1, 2020 - Dec 9, 2021, so approximately over a year of sales.
- customer_id: Missing values and incorrect type
- description: 21 unique products?
- stock_code: 20 unique, should be the same number as description?
- invoice_no: Missing values and incorrect type
- Quantity min is -321 (assuming returns) and max is 717
- Sales min is -4,911 (assuming returns) and max is 38,970

### Customers Table

In [9]:
df_customers.head()

,customer_id,order_city,order_postal,order_state,latitude,longitude
0,18287,Rock Springs,82901-7321,WY,41.5869,-109.2048
1,18283,JACKSON,83001-9460,WY,43.4800,-110.7618
2,18282,Greybull,82426,WY,44.4892,-108.0562
3,18281,EVANSTON,82930-4706,WY,41.2680,-110.9633
4,18280,CHEYENNE,82001-7901,WY,41.1400,-104.8202


In [10]:
customers_details = df_details(df_customers)
customers_details

DataFrame Shape: (4372, 6)



,Data Type,Count,Missing,Missing (%),Mean,Min,Median,Max,# Unique
customer_id,int64,4372,0,0.00,15300.0,12346.0,15300.0,18287.0,4372
order_city,object,4372,0,0.00,NaN,NaN,NaN,NaN,2665
order_postal,object,4372,0,0.00,NaN,NaN,NaN,NaN,4253
order_state,object,4372,0,0.00,NaN,NaN,NaN,NaN,212
latitude,float64,4274,98,2.24,38.0,18.0,39.0,61.0,2407
longitude,float64,4274,98,2.24,-92.0,-158.0,-87.0,-66.0,2417


<b>CUSTOMERS OBSERVATIONS:</b>

- customer_id: There seems to be no duplicate values
    - This will be the KEY to join with sales table
- order_city: Need to check formatting/standardization
- order_postal: Need to check formatting/standardization
- order_state: Assuming this is US sales, why do we have 212 unique values? Probably a formatting issue
- latitude/longitude: Missing 98 values

### Products Table

In [11]:
df_products.head()

,stock_code,weight,landed_cost,shipping_cost_1000_mile,description,category
0,22423,40.0,35.0,20.0,Taste of the Wild High Prairie Grain-Free Dry ...,Food
1,85123A,2.0,6.0,5.0,Dog and Puppy Pads,Disposables
2,47566,6.0,18.0,12.0,"Memory Foam Pet Beds for Small, Medium, and La...",Disposables
3,85099B,2.0,5.0,4.0,Earth Rated Dog Poop Bags,Disposables
4,23084,2.0,6.0,6.0,Templation Soft Cat Treats,Pet Food


In [12]:
products_details = df_details(df_products)
products_details

DataFrame Shape: (20, 6)



,Data Type,Count,Missing,Missing (%),Mean,Min,Median,Max,# Unique
stock_code,object,20,0,0.0,NaN,NaN,NaN,NaN,20
weight,float64,20,0,0.0,5.0,0.0,2.0,40.0,9
landed_cost,float64,20,0,0.0,14.0,2.0,10.0,36.0,15
shipping_cost_1000_mile,float64,20,0,0.0,7.0,2.0,6.0,20.0,9
description,object,20,0,0.0,NaN,NaN,NaN,NaN,20
category,object,20,0,0.0,NaN,NaN,NaN,NaN,7


<b>PRODUCTS OBSERVATIONS:</b>

- stock_code: 20 unique values, same value as description
    - This will be the KEY to join with sales table
- weigth: 9 unique values ranging from 0 - 40, if whole numbers change to int
- landed_cost: 15 unique values ranging from 2 - 36, if whole numbers change to int
- shipping_cost_1000_mile: 9 unique values ranging from 2 - 20, if whole numbers change to int
    - Same unique values as weight, compare to see if they are related
- description: 20 unique values, inline with stock_code
- category: 7 unique values

### State Region table

In [13]:
df_state_region.head()

,order_state,state,region
0,AE,AE,Other
1,ak,AK,Other
2,AK,AK,Other
3,AL,AL,East
4,ALABAMA,AL,East


In [14]:
state_details = df_details(df_state_region)
state_details

DataFrame Shape: (192, 3)



,Data Type,Count,Missing,Missing (%),Mean,Min,Median,Max,# Unique
order_state,object,192,0,0.0,NaN,NaN,NaN,NaN,192
state,object,192,0,0.0,NaN,NaN,NaN,NaN,54
region,object,192,0,0.0,NaN,NaN,NaN,NaN,4


<b>STATE REGION OBSERVATIONS:</b>

- order_state: 192 unique values, need to check formatting/standardization
    - This will be the KEY to join with Customers table
- state: 54 unique values, need to check
- region: 4 unique values

Now that we have a basic understanding on the tables and their relationships, let's proceed to clean the data so we can join them later for EDA.

### DATA CLEANING - SALES

#### Sales - customer_id

In [15]:
# Checking if we have any of null customer ids with invoice that we could map
missing_customers = df_sales[df_sales.customer_id.isnull()].groupby('invoice_no')['customer_id'].count().to_list()
missing_customers[missing_customers != 0]

0

In [16]:
# Converting column to string
df_sales.customer_id = df_sales.customer_id.astype('str')

In [17]:
# If customer_id = 'nan' we'll do GUEST-invoice_no
# If not we'll return the customer_id and remove the '.0'
df_sales.customer_id = df_sales.apply(lambda row: 'Guest_' + str(row['invoice_no'])[:6] \
                      if (row['customer_id'] == 'nan') else row['customer_id'][:5], axis=1)

In [18]:
# Checking that it worked
df_sales.sample(10)

,transaction_date,customer_id,description,stock_code,invoice_no,quantity,sales,unit_price
2311,2021-01-18 15:19:00,Guest_541497,Sheba Perfect Portions Pat Wet Cat Food,22197,541497.0,1,9.78,9.78
16211,2021-09-12 09:26:00,14110,Pet Hair Remover,23203,566305.0,2,24.96,12.48
7424,2021-05-05 10:22:00,12877,Indoor Pet Camera,22502,551895.0,3,89.10,29.70
7951,2021-05-10 15:06:00,Guest_552671,Milk-Bone MaroSnacks Dog Treats with Real Bone,22386,552671.0,1,24.78,24.78
18262,2021-10-06 11:48:00,13090,Pet Odor Eliminator,84879,569803.0,2,20.28,10.14
16083,2021-09-09 12:24:00,17001,Earth Rated Dog Poop Bags,85099B,566172.0,4,49.92,12.48
17844,2021-09-30 17:22:00,Guest_569202,ChomChom Pet Hair Remover - Reusable Cat and D...,22114,569202.0,1,49.74,49.74
21849,2021-11-14 12:56:00,12584,Rechargeable Pet Nail Grinder,22960,NaN,0,0.00,25.50
19923,2021-10-31 14:48:00,18257,Templation Soft Cat Treats,23084,573587.0,1,12.48,12.48
20311,2021-11-01 09:01:00,13461,Milk-Bone MaroSnacks Dog Treats with Real Bone,22386,573746.0,5,62.40,12.48


#### Sales - description

In [19]:
# Descriptions that are in SALES, but not in PRODUCT
for item in df_sales.description.unique():
    if item not in df_products.description.unique():
        print(item)

Sheba Perfect Portions Pat Wet Cat Food
Indoor Pet Camera (Wi-Fi)


In [20]:
df_sales[df_sales.description == 'Sheba Perfect Portions Pat Wet Cat Food']['description'].count()
# On PRODUCTS: Sheba Perfect Portions Paté Wet Cat Food

1451

In [21]:
df_sales[df_sales.description == 'Indoor Pet Camera (Wi-Fi)']['description'].count()
# On PRODUCTS: Indoor Pet Camera

53

In [22]:
# Descriptions that are in PRODUCTS, but not in SALES
for item in df_products.description.unique():
    if item not in df_sales.description.unique():
        print(item)

Sheba Perfect Portions Paté Wet Cat Food


In [23]:
df_sales[df_sales.description == 'Sheba Perfect Portions Paté Wet Cat Food']['description'].count()

0

In [24]:
df_sales[df_sales.description == 'Indoor Pet Camera']['description'].count()

426

NOTES: 
<br>There are 2 descriptions in SALES that are not in PRODUCTS

<br>Sales has:
<br>- 1451 sales with 'Sheba Perfect Portions Pat Wet Cat Food'
<br>- 0 sales with 'Sheba Perfect Portions Paté Wet Cat Food'

<br>Sales has:
<br>- 53 sales with 'Indoor Pet Camera (Wi-Fi)'
<br>- 426 sales with 'Indoor Pet Camera'

In [25]:
# Replacing descriptions with correct name from PRODUCTS
df_sales.loc[df_sales.description == 'Sheba Perfect Portions Pat Wet Cat Food','description'] = \
'Sheba Perfect Portions Paté Wet Cat Food'

In [26]:
# Checking that the description has been updated
df_sales[df_sales.description == 'Sheba Perfect Portions Paté Wet Cat Food']['description'].count()

1451

In [27]:
# Assuming PRODUCT should be the source of truth, we'll consolidate all sales to have description 'Indoor Pet Camera'
# instead of 'Indoor Pet Camera (Wi-Fi)'
df_sales.loc[df_sales.description == 'Indoor Pet Camera (Wi-Fi)','description'] = 'Indoor Pet Camera'

In [28]:
# Checking that the description has been updated (426 + 53) = 479
df_sales[df_sales.description == 'Indoor Pet Camera']['description'].count()

479

#### Sales - stock_code

In [29]:
len(df_sales.stock_code.unique()), len(df_products.stock_code.unique())

(20, 20)

In [30]:
# Checking if codes from SALES are not in PRODUCTS
for code in df_sales.stock_code.unique():
    if code not in df_products.stock_code.unique():
        print(code)

#### Sales - invoice_no: 

In [31]:
# We have 661 records without invoice_no
df_sales[df_sales.invoice_no.isnull()]['customer_id'].count()

661

In [32]:
# Converting column to string
df_sales.invoice_no = df_sales.invoice_no.astype('str')

In [33]:
# If null fill with 'no invoice'
# If not we'll return the invoice id and remove the '.0'
df_sales.invoice_no = df_sales.apply(lambda row: 'no invoice' if (row['invoice_no'] == 'nan') else row['invoice_no'][:6], axis=1)

In [34]:
df_sales[df_sales.invoice_no == 'no invoice'].sample(5)

,transaction_date,customer_id,description,stock_code,invoice_no,quantity,sales,unit_price
24408,2021-12-05 11:02:00,16360,Sheba Perfect Portions Paté Wet Cat Food,22197,no invoice,0,0.0,5.10
13827,2021-08-04 10:36:00,12476,Taste of the Wild High Prairie Grain-Free Dry ...,22423,no invoice,0,0.0,76.50
5004,2021-03-17 13:55:00,17511,Rechargeable Pet Nail Grinder,22960,no invoice,0,0.0,22.50
9205,2021-05-24 17:32:00,14194,"Memory Foam Pet Beds for Small, Medium, and La...",47566,no invoice,0,0.0,29.70
12795,2021-07-19 12:26:00,12415,Earth Rated Dog Poop Bags,85099B,no invoice,0,0.0,12.48


In [35]:
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25065 entries, 0 to 25064
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   transaction_date  25065 non-null  datetime64[ns]
 1   customer_id       25065 non-null  object        
 2   description       25065 non-null  object        
 3   stock_code        25065 non-null  object        
 4   invoice_no        25065 non-null  object        
 5   quantity          25065 non-null  int64         
 6   sales             25065 non-null  float64       
 7   unit_price        25065 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 1.5+ MB


### DATA CLEANING - CUSTOMERS

#### Customers - order_city

In [36]:
# Number of cities
df_customers.order_city.nunique()

2665

In [37]:
# Formatting city as Title to standardize
df_customers.order_city = df_customers.order_city.apply(lambda city: city.title())

In [38]:
# Number dropped from 2665 to 2231
df_customers.order_city.nunique()

2231

In [39]:
df_customers.order_city.value_counts().head(10)

New York         58
Brooklyn         40
Chicago          36
Los Angeles      29
Houston          29
Miami            28
San Francisco    27
San Diego        25
Seattle          21
Dallas           21
Name: order_city, dtype: int64

#### Customers - order_postal

In [40]:
df_customers.order_postal.nunique()

4253

In [41]:
# At first glance they look normal, just different if they include the additional 4 digits
# These last 4 digits represent specific delivery routes within delivery areas.
# During analysis, we also noticed some zip codes with only 3 or 4 digits,
# Doing a bit of research we found out these are normal in some states and we need to pad those with 0s on the left
df_customers.order_postal.sample(10)

1138    43213-2852
766          57010
1686    08512-2611
3424          6238
332     22046-3626
1124         45215
3993    93720-3855
1845         28409
1316    12564-3423
105     53504-9725
Name: order_postal, dtype: object

In [42]:
# Different lenghts of postal codes
df_customers.order_postal.apply(lambda postal: len(postal)).unique()

array([10,  5,  4,  3], dtype=int64)

In [43]:
df_customers['order_postal_5_digits'] = df_customers.order_postal.apply(lambda postal: postal[:5] if (len(postal) >= 5) else postal.zfill(5) )

In [44]:
# Checking it only has 5 digits
df_customers.order_postal_5_digits.apply(lambda postal: len(postal)).unique()

array([5], dtype=int64)

In [46]:
# Leaving only one column
df_customers.drop(columns='order_postal', axis=1, inplace=True)
df_customers.rename(columns={'order_postal_5_digits':'order_postal'}, inplace=True)

In [47]:
df_customers.sample(5)

,customer_id,order_city,order_state,latitude,longitude,order_postal
3658,13282,Santa Cruz,CA,36.9746,-122.0294,95060
3720,13198,San Francisco,CA,37.7648,-122.4630,94114
2568,14782,Versailles,KENTUCKY,38.0526,-84.7299,40383
1865,15728,Raleigh,NC,35.7804,-78.6391,27609
798,17193,Fort Mill,SC,35.0074,-80.9451,29715


#### Customers - order_state

In [48]:
# We are getting 212 states, but there should be only 50 states - we need to standardize
df_customers.order_state.nunique()

212

In [49]:
# Creating a list of US states abbreviations from https://usastatescode.com/state-array-json
states = ['AL','AK','AZ','AR','CA','CO','CT','DE','FL','GA','HI','ID','IL','IN','IA','KS','KY','LA','ME','MD','MA','MI','MN','MS','MO','MT','NE','NV','NH','NJ','NM','NY','NC','ND','OH','OK','OR','PA','RI','SC','SD','TN','TX','UT','VT','VA','WA','WV','WI','WY']

In [50]:
# We have a combination of full names, upper/lower case and abbreviations
df_customers.order_state.unique()[:40]

array(['WY', 'WV', 'Wisconsin', 'WI', 'wi', 'Wi', 'WEST VIRGINIA',
       'WASHINGTON', 'Washington', 'WA', 'wa', 'Wa', 'VT', 'VIRGINIA',
       'Virginia', 'VERMONT', 'Va.', 'VA', 'va', 'Va', 'UTAH', 'Utah',
       'UT', 'USVI', 'Tx', 'TX', 'tx', 'TN', 'tn', 'Texas', 'TEXAS', 'PA',
       'texas', 'Tennessee', 'TENNESSEE', 'South Carolina',
       'SOUTH CAROLINA', 'SD', 'SC', 'sc'], dtype=object)

In [51]:
# Lucky for us, the region table has most of these mappings
# We'll merge with the region table to get the values using left to make sure we keep all customers
df_customers = df_customers.merge(df_state_region, on='order_state', how='left')

In [54]:
# We get 54, but there are only 50 US states
# We see we also have some nan
df_customers.state.nunique(), df_customers.state.unique()

(54,
 array(['WY', 'WV', 'WI', 'WA', 'VT', 'VA', nan, 'UT', 'VI', 'TX', 'TN',
        'PA', 'SC', 'SD', 'RI', 'PR', 'OR', 'OK', 'OH', 'NY', 'NV', 'NC',
        'NM', 'NJ', 'NH', 'NE', 'ND', 'MT', 'MS', 'MO', 'MN', 'MI', 'ME',
        'MD', 'MA', 'LA', 'KY', 'KS', 'IA', 'IN', 'IL', 'ID', 'HI', 'GA',
        'FL', 'DC', 'DE', 'CT', 'CO', 'CA', 'AZ', 'AK', 'AR', 'AL', 'AE'],
       dtype=object))

In [55]:
# Which ones are not US states?
for state in df_customers.state.unique():
    if state not in states:
        print(state, end=' ')

nan VI PR DC AE 

In [56]:
# PR is Puerto Rico, DC is Washington DC, so we'll look at the other 2
df_customers[df_customers.state.isin(['VI','AE'])]['order_state'].count()

8

In [57]:
# St.Croix is in the Virgin Islands (VI)
# APO is Army Post Office,  FPO is Fleet Post Office
df_customers[df_customers.state.isin(['VI','AE'])]

,customer_id,order_city,order_state,latitude,longitude,order_postal,state,region
405,17727,St.Croix,USVI,NaN,NaN,00851,VI,Other
4271,12471,Fpo,AE,NaN,NaN,09645,AE,Other
4272,12468,Fpo,AE,NaN,NaN,09618,AE,Other
4273,12465,Apo,AE,NaN,NaN,09021,AE,Other
4274,12464,Apo,AE,NaN,NaN,09811,AE,Other
4275,12463,Apo,AE,NaN,NaN,09354,AE,Other
4276,12462,Apo,AE,NaN,NaN,09330,AE,Other
4277,12461,Apo,AE,NaN,NaN,09250,AE,Other


In [58]:
# Checking the nan
df_customers.state.isnull().sum()

22

In [59]:
# Values for in the order_state column for the null states
df_customers[df_customers.state.isnull()]['order_state'].unique()

array(['VERMONT', 'ri', 'Ne', 'N.C.', 'mo', 'Mississippi', 'Md', 'Ks',
       'Kansas', 'IL.', 'Ia', 'georgia', 'ga', 'florida', 'Ct', 'Az.',
       'Az', 'ALASKA', 'Alabama', 'Al'], dtype=object)

In [60]:
# Creating dict with State and abbreviation mapping
states_dict = { "Alabama": "AL", "Alaska": "AK", "American Samoa": "AS", "Arizona": "AZ", "Arkansas": "AR", "California": "CA", "Colorado": "CO", "Connecticut": "CT", "Delaware": "DE", "District Of Columbia": "DC", "Federated States Of Micronesia": "FM", "Florida": "FL", "Georgia": "GA", "Guam": "GU", "Hawaii": "HI", "Idaho": "ID", "Illinois": "IL", "Indiana": "IN", "Iowa": "IA", "Kansas": "KS", "Kentucky": "KY", "Louisiana": "LA", "Maine": "ME", "Marshall Islands": "MH", "Maryland": "MD", "Massachusetts": "MA", "Michigan": "MI", "Minnesota": "MN", "Mississippi": "MS", "Missouri": "MO", "Montana": "MT", "Nebraska": "NE", "Nevada": "NV", "New Hampshire": "NH", "New Jersey": "NJ", "New Mexico": "NM", "New York": "NY", "North Carolina": "NC", "North Dakota": "ND", "Northern Mariana Islands": "MP", "Ohio": "OH", "Oklahoma": "OK", "Oregon": "OR", "Palau": "PW", "Pennsylvania": "PA", "Puerto Rico": "PR", "Rhode Island": "RI", "South Carolina": "SC", "South Dakota": "SD", "Tennessee": "TN", "Texas": "TX", "Utah": "UT", "Vermont": "VT", "Virgin Islands": "VI", "Virginia": "VA", "Washington": "WA", "West Virginia": "WV", "Wisconsin": "WI", "Wyoming": "WY" }

In [61]:
# If order_state is abbreviated, removing periods and doing uppercase
# If not mapping the state abbreviation from the our dictionary
df_customers.loc[df_customers.state.isnull(), 'state'] = df_customers[df_customers.state.isnull()]['order_state'].\
                                                                apply(lambda state: state.replace('.','').upper() \
                                                                  if len(state) < 5 else states_dict.get(state.title()))

In [62]:
# Checking that we don't have more nulls for state
df_customers.state.isnull().sum(), df_customers.state.nunique()

(0, 54)

In [63]:
# Dropping order_state column
df_customers = df_customers.drop(columns='order_state')

In [64]:
# Renaming to original column name
df_customers.rename(columns={'state':'order_state'}, inplace=True)

#### Customers - Latitude/Longitude

In [65]:
# 98 rows without latitude or longitude
df_customers[(df_customers.latitude.isnull()) | (df_customers.longitude.isnull())]['customer_id'].count()

98

In [66]:
# Creating reference to obtain lat/long from zipcode
zip_long_lat = pd.read_csv('zip_lat_long.txt', dtype={'zip_code':str,'latitude':float,'longitude':float}, skiprows=1,\
                          names=['zipcode','latitude','longitude'])

In [67]:
# Converting zipcode to object and padding with zeros
zip_long_lat.zipcode = zip_long_lat.zipcode.apply(lambda zipcode: str(zipcode).zfill(5))

In [68]:
zip_long_lat.head(2)

,zipcode,latitude,longitude
0,00601,18.180555,-66.749961
1,00602,18.361945,-67.175597


In [71]:
# Merging customers to obtain missing lat/long
df_customers = df_customers.merge(zip_long_lat, left_on='order_postal', right_on='zipcode', how='left',\
                                  suffixes=(None,'_zip'))
df_customers.head(2)

,customer_id,order_city,latitude,longitude,order_postal,order_state,region,zipcode,latitude_zip,longitude_zip
0,18287,Rock Springs,41.5869,-109.2048,82901,WY,West,82901,41.379272,-108.978176
1,18283,Jackson,43.4800,-110.7618,83001,WY,West,83001,43.394562,-110.629837


In [89]:
# Round lat/long to 4 decimals (as original) for consistency
df_customers.latitude = df_customers.latitude.apply(lambda lat: round(lat, 4))
df_customers.longitude = df_customers.longitude.apply(lambda long: round(long, 4))

In [72]:
# Copying values from our merge to the columns with missing values
df_customers.latitude.fillna(df_customers.latitude_zip, inplace=True)
df_customers.longitude.fillna(df_customers.longitude_zip, inplace=True)

In [73]:
# We still have some missing lat/long, but they seem to be FPO (Fleet Post Office - Navy), APO (Army Post Office)
df_customers[df_customers.latitude.isnull()]

,customer_id,order_city,latitude,longitude,order_postal,order_state,region,zipcode,latitude_zip,longitude_zip
4271,12471,Fpo,NaN,NaN,09645,AE,Other,NaN,NaN,NaN
4272,12468,Fpo,NaN,NaN,09618,AE,Other,NaN,NaN,NaN
4273,12465,Apo,NaN,NaN,09021,AE,Other,NaN,NaN,NaN
4274,12464,Apo,NaN,NaN,09811,AE,Other,NaN,NaN,NaN
4275,12463,Apo,NaN,NaN,09354,AE,Other,NaN,NaN,NaN
4276,12462,Apo,NaN,NaN,09330,AE,Other,NaN,NaN,NaN
4277,12461,Apo,NaN,NaN,09250,AE,Other,NaN,NaN,NaN


In [74]:
# Let's use a number just to indicate these are NaN, instead of leaving them as Nan
df_customers.loc[np.isnan(df_customers.latitude), 'latitude'] = 0.0
df_customers.loc[np.isnan(df_customers.longitude), 'longitude'] = 0.0

In [75]:
df_customers[df_customers.latitude.isnull()]

,customer_id,order_city,latitude,longitude,order_postal,order_state,region,zipcode,latitude_zip,longitude_zip


In [76]:
# Cleaning up temporary columns we created
df_customers.drop(columns=['zipcode','latitude_zip','longitude_zip'], axis=1, inplace=True)

In [90]:
df_customers.head(2)

,customer_id,order_city,latitude,longitude,order_postal,order_state,region
0,18287,Rock Springs,41.5869,-109.2048,82901,WY,West
1,18283,Jackson,43.4800,-110.7618,83001,WY,West


### DATA CLEANING - PRODUCTS

#### Products - weight

In [79]:
# We see 9 unique weights, we'll leave them as float for now
df_products.weight.value_counts().sort_index()

0.5     3
1.0     6
2.0     5
5.0     1
6.0     1
7.0     1
10.0    1
15.0    1
40.0    1
Name: weight, dtype: int64

In [80]:
df_products.weight.min(), df_products.weight.max()

(0.5, 40.0)

#### Products - landed_cost

In [81]:
# We see 15 unique landed costs, we'll leave them as float for now
df_products.landed_cost.value_counts().sort_index()

2.5     1
4.0     2
5.0     1
6.0     3
8.0     2
9.0     1
12.0    1
15.0    1
18.0    2
20.0    1
22.0    1
23.0    1
30.0    1
35.0    1
36.0    1
Name: landed_cost, dtype: int64

In [82]:
df_products.landed_cost.min(), df_products.landed_cost.max()

(2.5, 36.0)

#### Products - shipping_cost_1000_mile

In [83]:
# We see 9 unique landed costs, we'll leave them as float for now
df_products.shipping_cost_1000_mile.value_counts().sort_index()

2.5     1
4.0     2
5.0     6
6.0     2
7.0     3
9.0     2
10.0    2
12.0    1
20.0    1
Name: shipping_cost_1000_mile, dtype: int64

In [84]:
df_products.shipping_cost_1000_mile.min(), df_products.shipping_cost_1000_mile.max()

(2.5, 20.0)

#### Products - category

In [85]:
df_products.category.value_counts()

Food                6
Disposables         4
Grooming            4
Supplements         2
Electronics         2
Pet Food            1
Cleanig Supplies    1
Name: category, dtype: int64

### Exporting clean data

In [86]:
# Exporting to our local instance of Postgres
df_sales.to_sql('sales', con=engine, if_exists='replace', index=False)
df_products.to_sql('products', con=engine, if_exists='replace', index=False)
df_customers.to_sql('customers', con=engine, if_exists='replace', index=False)

372